In [1]:
import importlib
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *

# CoastalWQL
This is the python notebook for the workflow process as follows:

![workflow]("workflow.jpg")

### Indicate the directory for storing processed images

## Inputs

#### Required inputs

In [10]:
image_fp = r"C:\Users\PAKHUIYING\Documents\image_processing\10_11_2021\11-34-17RES1280_EXP0047_GAN0030_BIT08_2EXP0000_GAN0010_RES1280_BIT08"
spectrometer_fp = r"C:\Users\PAKHUIYING\Documents\image_processing\10_11_2021\11_10_2021"
fp_store = "outputs"
prefix = "2021_11_10_11-34-17"
height = 58 #in meters
gps_index_txt = r"inputs\gps_index_2021_11_10_11-34-17.txt"
slider = 1400 #in milliseconds (time delay correction)
mask_checkbox = False
classify_checkbox = False
noise_checkbox = True
sunglint_checkbox = False
predict_checkbox = False
corrected_fp = r"inputs\corrected_indices.json" #corrected image indices (optional)
corrected_indices = None
line_start = 0
line_stop = 1
r,g,b = 38,23,15

with open(gps_index_txt, "r") as f:
    gps_indices = f.readlines()
gps_indices = [int(i.replace('\n','')) for i in gps_indices]
gps_indices.sort()

#------optional (can comment out)--------
with open(corrected_fp, "r") as read_file:
    corrected_indices = json.load(read_file)
corrected_indices = [(i['start'],i['stop']) for i in corrected_indices]


#### Sun glint correction inputs

In [3]:
sunglint_json_fp = r"inputs\sunglint_correction_2021_11_10_11-34-17.txt"
with open(sunglint_json_fp,"r") as cf:
    bbox = json.load(cf)

#### WQL inputs

In [9]:
wql_csv = r"inputs\wql.csv"
tss_df = pd.read_csv(wql_csv,engine='python')
tss_lat = tss_df['GPS Latitude ()'].tolist()
tss_lon = tss_df['GPS Longitude ()'].tolist()
tss_measurements = tss_df['Turbidity (FNU)'].tolist()
tss_df.head()

,DATE,TIME,SITE,DATA ID,GPS Latitude (),GPS Longitude (),Barometer (mmHg),Cond (µS/cm),Sp Cond (µS/cm),Sal (psu),nLFCond (µS/cm),TDS (mg/L),Temp (F),Resistivity (ohms-cm),Sigma-T (s t),Sigma (s),Turbidity (FNU),TSS (mg/L)
0,11/10/2021,11:34:06 am,Flight2,NaN,1.23614,103.63084,756.8,51401.3,47169.2,30.54,46701.3,30660,85.5,19.5,18.5,18.5,6.01,0
1,11/10/2021,11:34:07 am,Flight2,NaN,1.23613,103.63084,756.8,51401.6,47169.2,30.54,46701.3,30660,85.5,19.5,18.5,18.5,6.07,0
2,11/10/2021,11:34:08 am,Flight2,NaN,1.23613,103.63083,756.9,51401.7,47169.2,30.54,46701.3,30660,85.5,19.5,18.5,18.5,6.11,0
3,11/10/2021,11:34:09 am,Flight2,NaN,1.23612,103.63082,756.8,51401.7,47169.3,30.54,46701.3,30660,85.5,19.5,18.5,18.5,6.15,0
4,11/10/2021,11:34:10 am,Flight2,NaN,1.23612,103.63081,756.9,51401.5,47169.2,30.54,46701.3,30660,85.5,19.5,18.5,18.5,6.23,0


#### Image segmentation inputs

In [6]:
xgb_seg_fp = r"Models\image_segmentation\xgb_segmentation.json"
xgb_seg_model = load_xgb_segmentation_model(xgb_seg_fp)

#### Prediction inputs

In [ ]:
model_fp = r"Models\turbidity_prediction\XGB_corrected_trimmed100_1.json"
covariates_index_list = "4:15,33:38,43:46,53:54" #indices of covariates
covariates_index = covariates_index_list.replace(' ','').split(',')
covariates_index_list = []
for i in covariates_index:
    if ':' in i:
        c_start, c_end = i.split(':')
        covariates_index_list = covariates_index_list + list(range(int(c_start),int(c_end)+1))
    else:
        covariates_index_list.append(int(i))

In [7]:
gps_df = import_gps(image_fp)
unique_gps_df = get_unique_df(gps_df)
test_gps_index = gps_indices
indexes_list = gps_to_image_indices(unique_gps_df,image_fp,test_gps_index,slider)
reverse_boolean_list = rev_boolean_list(unique_gps_df,test_gps_index)

gps file path:C:\Users\PAKHUIYING\Documents\image_processing\10_11_2021\11-34-17RES1280_EXP0047_GAN0030_BIT08_2EXP0000_GAN0010_RES1280_BIT08/UAV/gps_11-34-47-860.csv


# Process images

### Save user-options

In [8]:
config_file = {'-PROCESSED_IMAGES-':fp_store,'-PREFIX-':prefix,'-IMAGE_FOLDER_FILEPATH-':image_fp,\
                '-SPECTRO_FILEPATH-':spectrometer_fp,\
                '-HEIGHT-':height,'-GPS_INDEX_TXT-':gps_index_txt,'-SLIDER-':slider,\
                '-LINE_START-':line_start,'-LINE_END-':line_stop,'rgb_bands':[r,g,b],\
                '-MASK_CHECKBOX-':mask_checkbox,'-CLASSIFY_CHECKBOX-':classify_checkbox,'-NOISE_CHECKBOX-':noise_checkbox,\
                '-SUNGLINT_CHECKBOX-':sunglint_checkbox,'-PREDICT_CHECKBOX-':predict_checkbox,\
                'corrected_indices':corrected_indices}

with open(join(fp_store,'config_file_{}.txt'.format(config_file['-PREFIX-'])),'w') as cf:
    json.dump(config_file,cf)

#### sun glint correction

In [ ]:
try:
    line_glint = bbox['glint']['line']
    start_i,end_i = indexes_list[line_glint]
    test_stitch_class = StitchHyperspectral(fp_store,config_file['-PREFIX-'],config_file['-IMAGE_FOLDER_FILEPATH-'],config_file['-SPECTRO_FILEPATH-'],\
        int(config_file['-HEIGHT-']),line_glint,start_i,end_i,\
        test_gps_index, unique_gps_df,reverse=reverse_boolean_list[line_glint])

    reflectance_glint = test_stitch_class.get_stitched_reflectance() #radiometrically corrected reflectance only for the image line where glint bbox is drawn on
except Exception as E:
    sunglint_json_fp = None
    reflectance_glint = None

In [11]:
for line_number in range(line_start, line_stop+1):
    start_i,end_i = indexes_list[line_number]
    test_stitch_class = StitchHyperspectral(fp_store,config_file['-PREFIX-'],config_file['-IMAGE_FOLDER_FILEPATH-'],config_file['-SPECTRO_FILEPATH-'],\
                        int(config_file['-HEIGHT-']),line_number,start_i,end_i,\
                        test_gps_index, unique_gps_df, destriping = config_file['-NOISE_CHECKBOX-'],\
                        reverse=reverse_boolean_list[line_number])
    test_stitch_class.view_pseudo_colour(r,g,b) #outputs rgb
    # #-----optional (can comment out)-------------
    # ##-----sunglint correction----
    # if config_file['-SUNGLINT_CHECKBOX-'] is True and sunglint_json_fp is not None and reflectance_glint is not None:
    #     sgc = SunglintCorrection(test_stitch_class,sunglint_json_fp,reflectance_glint)
    #     sgc.sunglint_correction_rgb()
    #     sgc_reflectance = sgc.sunglint_correction_reflectance()
    #     reflectance = None
    #     print("Performing sunglint correction on hyperspectral reflectances...")
    # else:
    #     print("No sunglint correction...")
    #     config_file['-SUNGLINT_CHECKBOX-'] = False
    #     sgc_reflectance = None
    ##-----masking--------------
    if config_file['-MASK_CHECKBOX-'] is True:
        print("Performing segmentation...")
        mask = test_stitch_class.get_mask(xgb_seg_model,type="XGBoost")
    else:
        print("Segmentation not conducted...")
        mask = None
    # ##-----radiometric correction--------------
    # if config_file['-RADIOMETRIC_CHECKBOX-'] is True:
    #     reflectance = test_stitch_class.get_stitched_reflectance()
    #     print("Getting radiometrically corrected reflectances...")
    # else:
    #     reflectance = test_stitch_class.get_stitched_uncorrected_reflectance()
    #     print("Getting uncorrected reflectances...")
    # ##---------extraction of spectral information
    # if tss_lat is not None:
    #     #extraction of spectral information
    #     test_stitch_class.get_reflectance_from_GPS(tss_lat,tss_lon,tss_measurements,\
    #         radius=2, mask = mask,\
    #         glint_corrected_reflectance=sgc_reflectance,reflectance = reflectance)
    gti = GeotransformImage(test_stitch_class,\
        mask = mask, classify = config_file['-CLASSIFY_CHECKBOX-'],\
        transform_predicted_image = config_file['-PREDICT_CHECKBOX-'],\
        sunglint_correction=config_file['-SUNGLINT_CHECKBOX-'])
    gti.geotransform_image()
    

Segmentation not conducted...
image obtained from gps indexes (86, 96): outputs\2021_11_10_11-34-17_rgb_image_line_00_3705_4145.tif
Flight angle is 59.20
Performing geotransformation...
Upper left corner coord: lat:1.2371850026029676, lon:103.63038231523548
Segmentation not conducted...
image obtained from gps indexes (112, 126): outputs\2021_11_10_11-34-17_rgb_image_line_01_4840_5454.tif
Flight angle is 59.71
Performing geotransformation...
Upper left corner coord: lat:1.2373836720331157, lon:103.63039596368444
